In [ ]:
import tensorflow as tf
import re
import gensim
import numpy as np
import pydot
import os
import json

from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Activation, Dropout, CuDNNLSTM, Input, Concatenate, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.utils import plot_model, to_categorical

from gensim.models.wrappers import FastText

#from AttentionWeightedAverage import AttentionWeightedAverage
from tensorflow.keras import backend as K
import tensorflow as tf


In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
pres = ['hoover', 'fdroosevelt', 'truman', 'hayes', 'adams', 'carter', 'madison', 'ford', 'cleveland', 'obama', 'harding', 'wilson', 'taylor', 'monroe', 'lincoln', '.ipynb_checkpoints', 'jefferson', 'vanburen', 'jackson', 'washington', 'polk', 'bush', 'gwbush', 'pierce', 'reagan', 'garfield', 'mckinley', 'coolidge', 'roosevelt', 'fillmore', 'johnson', 'harrison', 'taft', 'lbjohnson', 'jqadams', 'tyler', 'clinton', 'kennedy', 'eisenhower', 'nixon', 'arthur', 'grant', 'buchanan', 'bharrison']

file_train = ''
file_val = ''
file_test = ''

for root, dirs, files in os.walk("../1.DataPreparationResults", topdown=False):

    for name in files:
        file = os.path.join(root, name)
        
        if root.split('/')[2] in pres:
            text = open(file).read()
            
            if 'train' in file:
                file_train += text
            elif 'val' in file:
                file_val += text
            elif 'test' in file:
                file_test += text 

# Write to the 2.GPT2 XLNET folder
dir_split = "../1.DataPreparationResults/combined_data"
open(f"{dir_split}/train.txt", 'w').write(file_train)
open(f"{dir_split}/val.txt", 'w').write(file_val)
open(f"{dir_split}/test.txt", 'w').write(file_test)

In [ ]:
# This essentially is to produce a unique words list from all the text before splitting into sentences

def google_preprocess(file):
    file2 = re.sub('\d', '#', file)
    file2 = re.sub(' a ', ' A ', file2)
    file2 = re.sub(' and ', ' And ', file2)
    file2 = re.sub(' of ', ' Of ', file2)
    file2 = re.sub(' to ', ' To ', file2)
    # Add spaces around <speech_sep>
    # Create a set of all words in file.txt but remove <speech_sep>
    unique_words = set(file2.replace("<speech_sep>", " <speech_sep> ").split())
    unique_words.remove("<speech_sep>")
    return file2, unique_words

file_train_google, unique_words_train = google_preprocess(file_train)
file_val_google, unique_words_val = google_preprocess(file_val)
file_test_google, unique_words_test = google_preprocess(file_test)

unique_words_all = unique_words_train.union(unique_words_val.union(unique_words_test))
print("total number of unique words: ",len(unique_words_all))

In [ ]:
x_len = 30
x_step = 1

def file_to_rolling_sentences(file):
    sentences = []
    sentences2 = []
    next_words = []
    list_words = []
    
    for speech in file.split("<speech_sep>"):
        list_words = speech.split()
        # I noticed the last speech has zero word 
        # because <speech_sep> is the last character
        if len(list_words) == 0:
            break
        
        # each row should have x_len + 1 (both input and target)
        for i in range(0,len(list_words)-x_len-1, x_step):
            sentences2 = [word for word in list_words[i: i + x_len + 1]]
            sentences.append(sentences2)
            
    return sentences

# train_sentences = file_to_sentences(file_train)
train_sentences = file_to_rolling_sentences(file_train_google)
val_sentences = file_to_rolling_sentences(file_val_google)

In [ ]:
def file_to_each_sentence(file):
    
    sentence_all = []
    
    for speech in file.split("<speech_sep>"):
        sentences = speech.split('.')
        
        for sentence in sentences:
            sentence_all.append(sentence.strip() + '.')
    
    return sentence_all

train_sentences = file_to_each_sentence(file_train_google)
val_sentences = file_to_each_sentence(file_val_google)

In [ ]:
number_words = 25
maxlength = 64

tokenizer = Tokenizer(nb_words=number_words)
tokenizer.fit_on_texts(train_sentences)
tokenized_sequences = tokenizer.texts_to_sequences(train_sentences)

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
index_word = new_dict = dict([(value, key) for key, value in word_index.items()]) 

In [ ]:
train_data = pad_sequences(tokenized_sequences, maxlen=maxlength)

In [ ]:
train_x = train_data[:, :-1]
train_y = train_data[:, -1]

In [ ]:
tokenized_val = tokenizer.texts_to_sequences(val_sentences)
val_data = pad_sequences(tokenized_val, maxlen=maxlength)
val_x = val_data[:, :-1]
val_y = val_data[:, -1]

### Prepare Word Embeddings

In [16]:
google_word_model = gensim.models.KeyedVectors.load_word2vec_format('../../test/GoogleNews-vectors-negative300.bin', binary=True)


In [ ]:
#get all the vectors
#pretrained_weights = google_word_model.wv.vectors

#word2idx
def word2idx(word):
    return google_word_model.wv.vocab[word].index
    

#idx2word
def idx2word(idx):
    return google_word_model.wv.index2word[idx]

#get vector of word
#google_word_model["Hi"]

In [ ]:
EMBEDDING_DIM = 300
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))

for word, i in word_index.items():
    embedding_vector = google_word_model[word] if word in google_word_model else None
    
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
# model = tf.keras.Sequential()

# inputs = Input(shape=(49,))

# embedding_layer = Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix], input_length=maxlength-1, trainable=False)(inputs)
# lstm1 = LSTM(units = 300, return_sequences = True)(embedding_layer)
# lstm2 = LSTM(units = 300, return_sequences = True)(lstm1)
# predictions = Dense(len(word_index), activation='softmax')(lstm2)


# #test = Concatenate(axis=1)([embedding_layer, lstm1, lstm2])
# #custom_layer = AttentionWeightedAverage(name='attention')(test)

# model = Model(inputs=inputs,outputs=predictions)

# # model.add(embedding_layer)
# # model.add(LSTM(units = 128, return_sequences = True))
# # model.add(LSTM(units = 128, return_sequences = True))
# # model.add(Dense(len(word_index), activation='softmax'))

# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# #print(model.summary())
# #plot_model(model)

In [ ]:
embedding_layer = Embedding(len(word_index)+1, EMBEDDING_DIM, weights=[embedding_matrix], input_length=maxlength-1, trainable=False)

model = tf.keras.Sequential()
model.add(embedding_layer)
model.add(LSTM(units = 300, return_sequences=True))
model.add(LSTM(units = 300))
#model.add(Flatten())
model.add(Dense(len(word_index), activation='softmax'))


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
plot_model(model)

In [ ]:
history = model.fit(train_x, train_y, epochs=10, batch_size = 64, validation_data=(val_x, val_y)) #, callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

In [23]:
def sample(preds, temperature=1.0):
    if temperature <= 0:
        return np.argmax(preds)
    
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)


def generate_next(text, num_generated=50):
    tokenized_sequences = [word_index[word] for word in text.lower().split() if word in word_index.keys()]
    #tokenized_sequences = [word2idx(word) for word in text.lower().split() if word in google_word_model]
    
    #maxlen = maxlength, was 33 before.
    padded_sequences = pad_sequences([tokenized_sequences], maxlen=maxlength+1)[:, 1:]
    
    #print(padded_sequences)
    
    generated_tokens = []
    
    for i in range(num_generated):
        #print(padded_sequences.shape)
        prediction = model.predict(x=padded_sequences)
        idx = sample(prediction[-1], temperature=0.9)
        
        generated_tokens.append(idx)
        
        tokenized_sequences = np.append(tokenized_sequences, idx)
        padded_sequences = pad_sequences([tokenized_sequences], maxlen=maxlength+1)[:, 1:]
        
        
    return ' '.join([index_word[idx] if idx != 0 else "</unk>" for idx in generated_tokens])
    
    #return ' '.join([idx2word(idx) for idx in padded_sequences[0] if idx != 0])

def on_epoch_end(epoch, _):
    #print('\nGenerating text after epoch: %d' % epoch)
    
    texts = ["There are two ways to love you"]
    
    for text in texts:
        sample = generate_next(text)
        print('%s... -> %s' % (text, sample))

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
generate_next("Hello How are you")

In [ ]:
with open('train_history_sentence_style_google_w2v_embedtrain_F_LSTM2x.json', 'w') as f:
    json.dump(str(history.history), f)

In [ ]:
model.save('train_history_sentence_style_google_w2v_embedtrain_F_LSTM2x.h5') 

In [14]:
model = tf.keras.models.load_model('train_history_sentence_style_google_w2v_embedtrain_F_LSTM2x.h5')

# Check its architecture
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 64, 300)           7125900   
_________________________________________________________________
lstm_25 (LSTM)               (None, 64, 300)           721200    


In [24]:
generate_next("Hello How are you")

'as of of of to of our the to not the in not that on that the our be and to a of the have this to of of that the the of for the of the in of with the of the for to the the to have a'

In [25]:
fh = open("LSTM-test-file.txt").read().splitlines()

In [26]:
output = open("LSTM-fixed-window-result.txt", 'w')

In [28]:
for i,line in enumerate(fh):
    print("###############################")
    prediction = generate_next(line)
    actual = fh[i+1]
    
    output.write(line + '|' + actual + '|' + prediction + '\n')
    print(prediction)
    print(actual)

###############################
the be to with the be we of in of not to to of by with of is to of the to of of the of of for our the in of the of for to the the will have which have the by are our of of in the
Through its mysterious channels we come to wider acquaintance with surroundings and men.
###############################
to the this of we it the to to that the and of the to in our to to of to that to we it a of a to that by the and to of the in that not to our the have have be to a are of with
The microphone for these few moments has been brought to the President s study in the East Wing of the White House.
###############################
of is of a of it to to that be the a the of the i of the of to </unk> this the the of and be of the of of to our </unk> and in the the a our this be the be of to the a that the
This room from which I speak was the scene of work and accomplishment of our Presidents for over a century.
###############################
to in of and will a the the

to to of a the our of a the a of of a of a to our a the be of and the and the of of are </unk> the of of of the the of by of with to this the in that the of of have on in
That is the reduction of arms.
###############################
of a a be as of to to the to a by our a with be to to of and of a this a of of our and and for on of of that is with a to this which is have this the for that of in for the
Some months ago I proposed to the world that we should further reduce and limit naval arms.
###############################
the to the have the by of of of for that that will have will and of the by and have this the of of to the by a the and a the a a a the is the the of by a the of of our a of to
Today we are engaged in a most hopeful discussion with other governments leading to this end.
###############################
of the of a and a a be a of the the of the the of this of the of to to of of that not have of for we of the of have the the the by the that the the of the it the and t

in are are in to the of of have to the the a as of of of and of are by our and which i to to to our for of of be for the of are of of that to in in the of to to the of for
In that case there was no Federal Reserve System.
###############################
of of have to to the is and and the our to to to the our in a of be a will our not the i and a of a and of the by have in of with the of our a the the as that of and that of
There was no way to acquaint of capital movement over the country , and the interest rates ran up to 300 percent.
###############################
to in to of the not our this in a the of to not to not of be and of to by of to the will of for our of it the with that a is our the to have a the it that in of a we of of
The result was to bring about a monetary panic in the entire country.
###############################
i to to of and by of that </unk> of this not are a of of the of be the of are be to the to and the our of the to is that on to of to of of the by our </

to of </unk> that and of the for and of a </unk> this be is be our the to that and in of the of the the to to of by by this the we is the the in of the the </unk> the the of of of of of
And that is about a summary of the whole situation as it stands at this moment.
###############################
a a a our </unk> to and the of this our the of our the that the the not the and of the of be of of to of that in that the of that the and by our of to not not that of the the the a i
.
###############################
we the of to the the of of to the a the with of not to by the for a of </unk> the to to a for and by of are is with of of a i in of a to of for to of we this of of of
We are dealing here with a psychological situation to a very considerable degree.
###############################
are the </unk> of the to the a of a the will this have of with the of of the the of the of have and to of with of the this of are it the </unk> of of with a which the and of the and the of the
It is a que

the of and to the a the have with in to the to i by the a with in will the not we of are of the of that on a the it a the </unk> in is for of of of the of the to in and i our
My Countrymen This occasion is not alone the administration of the most sacred oath which can be assumed by an American citizen.
###############################
that to of a of the a the are a are to our our in and of to of have the of </unk> to the of of of of a it of in the the of and we a the a this the to of by to to of the
It is a dedication and consecration under God to the highest office in service of our people.
###############################
of the be this by by this have and of a are with a the the of it the not the of of the to as and the to the is to are i to the </unk> and of the to and a the will this </unk> the to and
I assume this trust in the humility of knowledge that only through the guidance of Almighty Providence can I hope to discharge its ever increasing burdens.
###########################

...

to of the of the to have a of a </unk> will of of our the in of to and of the of the the of be of our </unk> be that of of the of our to this of the that of to to the a </unk> a by
We have as a Nation a definite duty to see that no deserving person in our country suffers from hunger or cold.
###############################
that of a to of a in our the i the of a the be the our for the a the i have that in of with and a of will which to of be to the a a that a have to of that that </unk> and our will
I therefore set up a more extensive organization to stimulate more intensive cooperation throughout the country.
###############################
to a of by the the of that and it the of of the to in of the the of of the the and as of to the our to this of have is not a of be by our the this is this the the of and we have
There has been a most gratifying degree of response , from governors , mayors , and other public officials , from welfare organizations , and from employers in concerns bot

of is to to the and a by to of in this a to a are the the and the our to a in will will of the have of which to to in the a the for the i is the the of to our a have for will
Our immediate problem is the increase of employment for the next six months , and new plans which do not produce such immediate result or which extend commitments beyond this period are not warranted.
###############################
of the in the of the the and of of by the and of our on the will to be to we the the the be of not of the to and the of of our the of is for the a and to to in of of a a
The enlarged rivers and harbors , public building , and highway plans authorized by the Congress last session , however , offer an opportunity for assistance by the temporary acceleration of construction of these programs even faster than originally planned , especially if the technical requirements of the laws which entail great delays could be amended in such fashion as to speed up acquirements of land and the lettin

is the the i of </unk> we it to is the and to and of of </unk> the this of of to a the our the in of and the the a the of with to the of not the the have be of </unk> we of of this the
Railway rates were reduced on feed and livestock in and out of the drought areas , and over 50 , 000 cars of such products have been transported under these reduced rates.
###############################
i and and it the in to a the of to to be of have the the to not our that of of in to of this in that of a to this of the of the the the a a are and for </unk> on be will we by
The Red Cross established a preliminary fund of 5 , 000 , 000 for distress relief purposes and established agencies for its administration in each county.
###############################
be of of by are the a of i the that the the </unk> a a we the and to of to our to of and of in a a have this our of of of the for of the be in to of that it the to that of
Of this fund less than 500 , 000 has been called for up to this time as the 

of </unk> the the to will of our the of of with our of this the of are of of to a a of of a of our the the the in of and the is of to the the to of it to to </unk> </unk> a to to
Our Army and Navy are being maintained at a high state of efficiency , under officers of high training and intelligence , supported by a devoted personnel of the rank and file.
###############################
this a of our our our have a of a to not we that not that our the a the to a of i a as this be our </unk> the a be the the to and the with the the </unk> a to </unk> the and on of to
The London naval treaty has brought important economies in the conduct of the Navy.
###############################
the of of a the it have the the the have of the have for will the the to the this to to a of of of we of </unk> that will the the the to the are the the of be this a and and not this the of
The Navy Department will lay before the committees of the Congress recommendations for a program of authorization of new co

KeyboardInterrupt: 